In [1]:
import os
import sys
import warnings
import scanpy as sc
import pandas as pd
import anndata as ad
import pickle
# import multiprocessing
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")
sys.path.append(os.getcwd())

In [4]:
# sc_adata = pd.read_table(paths['sc_path']+"/sc_data.tsv", sep='\t',index_col=0)
# ST_adata = pd.read_table(ST_path+"/MERFISH_ID{0}_data_{1}.tsv".format(idx1,idx2), sep='\t',index_col=0)
# st_adata = pd.read_table(paths['ST_path']+"/ST_data.tsv", sep='\t',index_col=0)
# sc_adata.fillna(0,inplace = True)
# sc_adata.to_csv(paths['sc_path']+"/sc_data.tsv", sep='\t')

In [ ]:
sc_adata

In [ ]:
ID_list = range(1,37)

In [5]:
idx1 = 1
idx2 = 0.01

In [6]:
load_test_groundtruth = False
use_marker_genes = False
external_genes = True
generate_new_pseudo_spots = True

# pseudo_spot_simulation_paras = {
#     'spot_num': 30000,
#     'min_cell_num_in_spot': 20,
#     'max_cell_num_in_spot': 100,
#     'generation_method': 'celltype',
#     'max_cell_types_in_spot': 4,   
# }

paths = {
    'sc_path': '/data/lyx/hubs/SpaTD/stdgcn/benchmark_data/MERFISH/sc_data',
    'ST_path': '/data/lyx/hubs/SpaTD/stdgcn/benchmark_data/MERFISH/ST_data',
    'output_path': '/data/lyx/hubs/SpaTD/stdgcn/benchmark/MERFISH/ID1/Bregma0.01/',
}
    
pseudo_spot_simulation_paras = {
    'spot_num': 30000,
    'min_cell_num_in_spot': 8,
    'max_cell_num_in_spot': 12,
    'generation_method': 'celltype',
    'max_cell_types_in_spot': 4,   
}
# paths = {
#     'sc_path': '/data/lyx/hubs/SpaTD/stdgcn/benchmark_data/seqFISH_plus/Dataset3_seqFISHplus_sometasensory/Cortex_allField_77spot/sc_data',
#     'ST_path': '/data/lyx/hubs/SpaTD/stdgcn/benchmark_data/seqFISH_plus/Dataset3_seqFISHplus_sometasensory/Cortex_allField_77spot/ST_data',
#     'output_path': '/data/lyx/hubs/SpaTD/stdgcn/benchmark/seqFISH_plus/Dataset3_Cortex_allField_77spot',
# }
fraction_pie_plot = True
cell_type_distribution_plot = True
n_jobs = -1
GCN_device = 'GPU'

sc_path = paths['sc_path']
ST_path = paths['ST_path']
output_path = paths['output_path']

if not os.path.exists(output_path):
    os.makedirs(output_path)
    
#sc_adata = sc.read_csv(sc_path+"/sc_data.tsv", delimiter='\t')
sc_adata = sc.read_csv(sc_path+"/MERFISH_ID{0}_{1}_sc_data.tsv".format(idx1,idx2), delimiter='\t')
sc_label = pd.read_table(sc_path+"/sc_label.tsv", sep = '\t', header = 0, index_col = 0, encoding = "utf-8")
sc_label.columns = ['cell_type']
sc_adata.obs['cell_type'] = sc_label['cell_type'].values

cell_type_num = len(sc_adata.obs['cell_type'].unique())
cell_types = sc_adata.obs['cell_type'].unique()

word_to_idx_celltype = {word: i for i, word in enumerate(cell_types)}
idx_to_word_celltype = {i: word for i, word in enumerate(cell_types)}

celltype_idx = [word_to_idx_celltype[w] for w in sc_adata.obs['cell_type']]
sc_adata.obs['cell_type_idx'] = celltype_idx
sc_adata.obs['cell_type'].value_counts()

Inhibitory                  13797
Mature_oligodendrocyte       6334
Excitatory                   3144
Immature_oligodendrocyte     1680
Microglia                     826
Astrocyte                     802
Ambiguous                     751
Endothelial                   529
Mural                         504
Ependymal                      66
Name: cell_type, dtype: int64

In [7]:
# with open(output_path+"/marker_genes.tsv", 'r') as f:
#     selected_genes = [line.rstrip('\n') for line in f]
selected_genes =sc_adata.var_names.to_list()

In [8]:
sc_path+"/MERFISH_ID{0}_{1}_sc_data.tsv".format(idx1,idx2)

'/data/lyx/hubs/SpaTD/stdgcn/benchmark_data/MERFISH/sc_data/MERFISH_ID1_0.01_sc_data.tsv'

In [9]:
spot_num = pseudo_spot_simulation_paras['spot_num']
min_cell_number_in_spot = pseudo_spot_simulation_paras['min_cell_num_in_spot']
max_cell_number_in_spot = pseudo_spot_simulation_paras['max_cell_num_in_spot']
max_cell_types_in_spot = pseudo_spot_simulation_paras['max_cell_types_in_spot']
generation_method = pseudo_spot_simulation_paras['generation_method']
n_jobs = 24

In [10]:
sc_exp =sc_adata.copy()

In [11]:
# cell_type_num = len(sc_adata.obs['cell_type'].unique())
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=n_jobs)

In [12]:
# args = [(sc_exp, min_cell_number_in_spot, max_cell_number_in_spot, max_cell_types_in_spot, generation_method) for i in range(spot_num)]

In [13]:
import numpy as np
import random
np.random.seed(0)

In [14]:
random.seed(0)

In [15]:
output_path

'/data/lyx/hubs/SpaTD/stdgcn/benchmark/MERFISH/ID1/Bregma0.01/'

In [18]:
%%time
pseudo_adata = pseudo_spot_generation(sc_adata,
                                      idx_to_word_celltype,
                                      spot_num = pseudo_spot_simulation_paras['spot_num'],
                                      min_cell_number_in_spot = pseudo_spot_simulation_paras['min_cell_num_in_spot'],
                                      max_cell_number_in_spot = pseudo_spot_simulation_paras['max_cell_num_in_spot'],
                                      max_cell_types_in_spot = pseudo_spot_simulation_paras['max_cell_types_in_spot'],
                                      generation_method = pseudo_spot_simulation_paras['generation_method'],
                                      n_jobs = n_jobs
                                      )
data_file = open(output_path+'/pseudo_ST_ID1_8_12_4.pkl','wb')
pickle.dump(pseudo_adata, data_file)
data_file.close()

######### have done  0.0 ############
######### have done  0.03333333333333333 ############
######### have done  0.06666666666666667 ############
######### have done  0.1 ############
######### have done  0.13333333333333333 ############
######### have done  0.16666666666666666 ############
######### have done  0.2 ############
######### have done  0.23333333333333334 ############
######### have done  0.26666666666666666 ############
######### have done  0.3 ############
######### have done  0.3333333333333333 ############
######### have done  0.36666666666666664 ############
######### have done  0.4 ############
######### have done  0.43333333333333335 ############
######### have done  0.4666666666666667 ############
######### have done  0.5 ############
######### have done  0.5333333333333333 ############
######### have done  0.5666666666666667 ############
######### have done  0.6 ############
######### have done  0.6333333333333333 ############
######### have done  0.66666666666666

In [160]:
mv /data/lyx/hubs/SpaTD/stdgcn/benchmark/MERFISH/ID1/Bregma0.26/pseudo_ST_8_12_4.pkl /data/lyx/hubs/SpaTD/stdgcn/benchmark/MERFISH/ID1/Bregma0.01/

In [113]:
!rm {output_path+'/pseudo_ST.pkl'}

In [17]:
def generate_a_spot(sc_exp, 
                    min_cell_number_in_spot, 
                    max_cell_number_in_spot,
                    max_cell_types_in_spot,
                    generation_method):
    if generation_method == 'cell':
        cell_num = random.randint(min_cell_number_in_spot, max_cell_number_in_spot)
        cell_list = list(sc_exp.obs.index.values)
        picked_cells = random.choices(cell_list, k=cell_num)
        return sc_exp[picked_cells]
    elif generation_method == 'celltype':
        cell_num = random.randint(min_cell_number_in_spot, max_cell_number_in_spot)
        cell_type_list = list(sc_exp.obs['cell_type'].unique())
        cell_type_num = random.randint(1, max_cell_types_in_spot)
        
        while(True):
            cell_type_list_selected = random.choices(sc_exp.obs['cell_type'].value_counts().keys(), k=cell_type_num)
            if len(set(cell_type_list_selected)) == cell_type_num:
                break
        sc_exp_filter = sc_exp[sc_exp.obs['cell_type'].isin(cell_type_list_selected)]
        
        picked_cell_type = random.choices(cell_type_list_selected, k=cell_num)
        picked_cells = []
        for i in picked_cell_type:
            data = sc_exp[sc_exp.obs['cell_type'] == i]
            cell_list = list(data.obs.index.values)
            picked_cells.append(random.sample(cell_list, 1)[0])
            
        return sc_exp_filter[picked_cells]
    else:
        print('generation_method should be "cell" or "celltype" ')

def pseudo_spot_generation(sc_exp, 
                           idx_to_word_celltype,
                           spot_num, 
                           min_cell_number_in_spot, 
                           max_cell_number_in_spot,
                           max_cell_types_in_spot,
                           generation_method,
                           n_jobs = -1
                          ):
    
    cell_type_num = len(sc_exp.obs['cell_type'].unique())
    
    # cores = multiprocessing.cpu_count()
    # if n_jobs == -1:
    #     pool = multiprocessing.Pool(processes=cores)
    # else:
    #     pool = multiprocessing.Pool(processes=n_jobs)
    # args = [(sc_exp, min_cell_number_in_spot, max_cell_number_in_spot, max_cell_types_in_spot, generation_method) for i in range(spot_num)]    
    pseudo_spots = []
    pseudo_spots_table = np.zeros((spot_num, sc_exp.shape[1]), dtype=float)
    pseudo_fraction_table = np.zeros((spot_num, cell_type_num), dtype=float)
    for i in range(spot_num):
        one_spot = generate_a_spot(sc_exp, 
                    min_cell_number_in_spot, 
                    max_cell_number_in_spot,
                    max_cell_types_in_spot,
                    generation_method)
        pseudo_spots.append(one_spot)
        pseudo_spots_table[i] = one_spot.X.sum(axis=0)
        for j in one_spot.obs.index:
            type_idx = one_spot.obs.loc[j, 'cell_type_idx']
            pseudo_fraction_table[i, type_idx] += 1
        if i %1000==0:
            print("######### have done ",i/spot_num,'############')

    pseudo_spots_table = pd.DataFrame(pseudo_spots_table, columns=sc_exp.var.index.values)
    pseudo_spots = ad.AnnData(X=pseudo_spots_table.iloc[:,:].values)
    pseudo_spots.obs.index = pseudo_spots_table.index[:]
    pseudo_spots.var.index = pseudo_spots_table.columns[:]
    type_list = [idx_to_word_celltype[i] for i in range(cell_type_num)]
    pseudo_fraction_table = pd.DataFrame(pseudo_fraction_table, columns=type_list)
    pseudo_fraction_table['cell_num'] = pseudo_fraction_table.sum(axis=1)
    for i in pseudo_fraction_table.columns[:-1]:
        pseudo_fraction_table[i] = pseudo_fraction_table[i]/pseudo_fraction_table['cell_num']
    pseudo_spots.obs = pseudo_spots.obs.join(pseudo_fraction_table)
        
    return pseudo_spots

In [11]:
# def generate_a_spot(sc_exp, 
#                     min_cell_number_in_spot, 
#                     max_cell_number_in_spot,
#                     max_cell_types_in_spot,
#                     generation_method):
#     if generation_method == 'cell':
#         cell_num = random.randint(min_cell_number_in_spot, max_cell_number_in_spot)
#         cell_list = list(sc_exp.obs.index.values)
#         picked_cells = random.choices(cell_list, k=cell_num)
#         return sc_exp[picked_cells]
#     elif generation_method == 'celltype':
#         cell_num = random.randint(min_cell_number_in_spot, max_cell_number_in_spot)
#         cell_type_list = list(sc_exp.obs['cell_type'].unique())
#         cell_type_num = random.randint(1, max_cell_types_in_spot)
        
#         while(True):
#             cell_type_list_selected = random.choices(sc_exp.obs['cell_type'].value_counts().keys(), k=cell_type_num)
#             if len(set(cell_type_list_selected)) == cell_type_num:
#                 break
#         sc_exp_filter = sc_exp[sc_exp.obs['cell_type'].isin(cell_type_list_selected)]
        
#         picked_cell_type = random.choices(cell_type_list_selected, k=cell_num)
#         picked_cells = []
#         for i in picked_cell_type:
#             data = sc_exp[sc_exp.obs['cell_type'] == i]
#             cell_list = list(data.obs.index.values)
#             picked_cells.append(random.sample(cell_list, 1)[0])
            
#         return sc_exp_filter[picked_cells]
#     else:
#         print('generation_method should be "cell" or "celltype" ')

# def pseudo_spot_generation(sc_exp, 
#                            idx_to_word_celltype,
#                            spot_num, 
#                            min_cell_number_in_spot, 
#                            max_cell_number_in_spot,
#                            max_cell_types_in_spot,
#                            generation_method,
#                            n_jobs = -1
#                           ):
    
#     cell_type_num = len(sc_exp.obs['cell_type'].unique())
    
#     cores = multiprocessing.cpu_count()
#     if n_jobs == -1:
#         pool = multiprocessing.Pool(processes=cores)
#     else:
#         pool = multiprocessing.Pool(processes=n_jobs)
#     args = [(sc_exp, min_cell_number_in_spot, max_cell_number_in_spot, max_cell_types_in_spot, generation_method) for i in range(spot_num)]
#     generated_spots = pool.starmap(generate_a_spot, tqdm(args, desc='Generating pseudo-spots'))
    
#     pseudo_spots = []
#     pseudo_spots_table = np.zeros((spot_num, sc_exp.shape[1]), dtype=float)
#     pseudo_fraction_table = np.zeros((spot_num, cell_type_num), dtype=float)
#     for i in range(spot_num):
#         one_spot = generated_spots[i]
#         pseudo_spots.append(one_spot)
#         pseudo_spots_table[i] = one_spot.X.sum(axis=0)
#         for j in one_spot.obs.index:
#             type_idx = one_spot.obs.loc[j, 'cell_type_idx']
#             pseudo_fraction_table[i, type_idx] += 1
#     pseudo_spots_table = pd.DataFrame(pseudo_spots_table, columns=sc_exp.var.index.values)
#     pseudo_spots = anndata.AnnData(X=pseudo_spots_table.iloc[:,:].values)
#     pseudo_spots.obs.index = pseudo_spots_table.index[:]
#     pseudo_spots.var.index = pseudo_spots_table.columns[:]
#     type_list = [idx_to_word_celltype[i] for i in range(cell_type_num)]
#     pseudo_fraction_table = pd.DataFrame(pseudo_fraction_table, columns=type_list)
#     pseudo_fraction_table['cell_num'] = pseudo_fraction_table.sum(axis=1)
#     for i in pseudo_fraction_table.columns[:-1]:
#         pseudo_fraction_table[i] = pseudo_fraction_table[i]/pseudo_fraction_table['cell_num']
#     pseudo_spots.obs = pseudo_spots.obs.join(pseudo_fraction_table)
        
#     return pseudo_spots